# ==========
# Simulate the underlying EHR

In [19]:
pacman::p_load( simmer )

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)


simmer installed



In [37]:
set.seed(1289)
customer <-
  simmer::trajectory( name = "Customer's path" ) %>%
  simmer::log_("Here I am") %>%
  simmer::timeout( 12 ) %>% # This is a hardcoded duration at node.
  simmer::log_("I must leave")

sim_bank <-
  simmer::simmer( name = "bank") %>%
  simmer::add_generator("Customer", customer, function() { c(0, rexp(4, 1/10), -1) } )

sim_bank %>% simmer::run( until = 400 )
sim_bank %>% simmer::get_mon_arrivals()

0: Customer0: Here I am
12: Customer0: I must leave
14.3114: Customer1: Here I am
26.3114: Customer1: I must leave
26.5588: Customer2: Here I am
35.8506: Customer3: Here I am
38.2706: Customer4: Here I am
38.5588: Customer2: I must leave
47.8506: Customer3: I must leave
50.2706: Customer4: I must leave


simmer environment: bank | now: 50.2705868901582 | next: 
{ Monitor: in memory }
{ Source: Customer | monitored: 1 | n_generated: 5 }

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
Customer0,0.00000,12.00000,12,TRUE,1
Customer1,14.31136,26.31136,12,TRUE,1
Customer2,26.55882,38.55882,12,TRUE,1
Customer3,35.85064,47.85064,12,TRUE,1
Customer4,38.27059,50.27059,12,TRUE,1


In [79]:

set.seed(1269)

customer <-
  trajectory("Customer's path") %>%
  log_("Here I am") %>%
  set_attribute("start_time", function() {now(bank)}) %>%
  seize("counter") %>%
  log_(function() {paste("Waited: ", now(bank) - get_attribute(bank, "start_time"))}) %>%
  # timeout(rexp(1, 1/12)) would generate a single random time and use it for
  # every arrival, whereas the following line generates a random time for each
  # arrival
  timeout(function() {rexp(1, 1/12)}) %>%
  release("counter") %>%
  log_(function() {paste("Finished: ", now(bank))})

bank <-
  simmer("bank") %>%
  add_resource("counter") %>%
  add_generator("Customer", customer, function() {c(0, rexp(4, 1/10), -1)})

bank %>% run(until = 400)
bank %>%
  get_mon_arrivals() %>%
  transform(waiting_time = end_time - start_time - activity_time)

0: Customer0: Here I am
0: Customer0: Waited:  0
3.68365: Customer1: Here I am
3.89529: Customer2: Here I am
4.91521: Customer0: Finished:  4.91521483175031
4.91521: Customer1: Waited:  1.23156707156024
10.961: Customer3: Here I am
12.4143: Customer4: Here I am
21.0552: Customer1: Finished:  21.055182798071
21.0552: Customer2: Waited:  17.1598927142276
65.3116: Customer2: Finished:  65.3115507844349
65.3116: Customer3: Waited:  54.3505873236467
80.1134: Customer3: Finished:  80.1133545241526
80.1134: Customer4: Waited:  67.6990166719428
93.7286: Customer4: Finished:  93.7285504163997


simmer environment: bank | now: 93.7285504163997 | next: 
{ Monitor: in memory }
{ Resource: counter | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: Customer | monitored: 1 | n_generated: 5 }

name,start_time,end_time,activity_time,finished,replication,waiting_time
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>,<dbl>
Customer0,0.000000,4.915215,4.915215,TRUE,1,0.000000
Customer1,3.683648,21.055183,16.139968,TRUE,1,1.231567
Customer2,3.895290,65.311551,44.256368,TRUE,1,17.159893
Customer3,10.960963,80.113355,14.801804,TRUE,1,54.350587
Customer4,12.414338,93.728550,13.615196,TRUE,1,67.699017
